# 导入模块

In [200]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False
import json
import warnings
warnings.filterwarnings("ignore")
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [201]:
clean = Data_Clean()

In [202]:
f_path="F:/myfile/p站数据/当月数据"
#f_path="F:/下班前订单数据"
#f_path="F:/订单日报/临时导出"

In [203]:
import os
excel_names = []
excel_date = []
for excel_name in os.listdir(f_path):
    # excel_names.append(excel_name)
    # excel_date.append(excel_name.split('.')[0])
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        excel_names.append(excel_name)
        excel_date.append(excel_name.split('.')[0])
print(excel_names)
print(excel_date)


['1001-1015.xlsx']
['1001-1015']


In [204]:
# df_list=[]
# for excel_name in excel_names:
#     # 读取每个excel到df
#     excel_path = f"{f_path}/{excel_name}"
#     print(excel_path)
#     df_split = pd.read_excel(excel_path,skiprows=0)
#     print(df_split)
#     df_list.append(df_split)
#     print(df_list)

# df = pd.concat(df_list,ignore_index=True)

In [205]:
df = pd.read_excel(f'F:/myfile/p站数据/当月数据/{excel_date[-1]}.xlsx')

In [206]:
df["下单日期"]=df["下单时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month

In [207]:
df = df[df['商品属性'].notnull()]

# 信息获取

In [208]:
df["拒绝理由"]=df["备注信息"].str.split("拒绝理由：").str[1]
df["拒绝理由"]=df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')
df["取消原因"]=df["备注信息"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["备注信息"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
#df["无法联系原因"]=df["备注信息"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["下单商品"].str.contains('99新')|df["下单商品"].str.contains('95新')|df["下单商品"].str.contains('准新'),"二手","全新")
df.loc[:,"租赁方案"]=np.where(df["商品属性"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["预授权占用自有资金"]>0,"部分免押","全免押")
df.loc[:,"是否使用优惠券"]=np.where((df["设置买断金额"]>df["总租金"])&(df["租赁方案"]=="租完即送"),"使用优惠券","未使用优惠券")
df.loc[:,"是否进行预授权"]=np.where(df["预授权占用自有资金"].isnull(),"未预授权","已预授权")

In [209]:
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["商品属性"]),axis=1)

In [210]:
def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["商品属性"]),axis=1)

In [211]:
df.loc[:,"进件"]=np.where((df["订单状态"]=="待支付")|(df["订单状态"]=="订单取消"),"未进件","进件")

# 渠道归属

In [212]:
df.loc[:,"来源渠道"]=df["来源渠道"].fillna("未知渠道")

In [213]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if '京东活动' in b:
        return '京享租'
    elif "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "苹果BD" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "支付宝直播" in a or '支付宝直播' in b or '支付宝直播商品' in b:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in b:
        return "生活号"
    elif "社群" in b or '支群' in b:
        return "支付宝社群"
    
    else :
        return a

In [214]:
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["活动名称"]),axis=1)

In [215]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["订单状态"].map(dict_status_code)

In [216]:
df.sort_values(by=["下单日期","状态编码"],inplace=True)

# 删除重复数据

In [217]:
#删除重复单号
df.drop_duplicates(subset=["单号"],inplace=True)
#删除身份证空值行
df.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['活动名称']=="线下门店3个月试行"].index,inplace=True)
#删除订单状态空值行
df.dropna(subset=["订单状态"],axis=0,inplace=True)
df.drop(df[df['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)

In [218]:
#删除重复订单
df.drop_duplicates(subset=["单号"],inplace=True)
df.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)

# 定义状态

In [219]:
def reject_type(a,b,c,d,e,f,g):
    if a is not np.nan and b=="未进件":
        return "前置拦截"
    elif g is np.nan and b=="未进件":
        return "进件前取消"
    elif a is not np.nan and b=="进件":
        return "机审强拒"
    elif c is not np.nan and"抖音非免押用户" in c:
        return "抖音非免押拒绝"
    elif a is np.nan and b=="进件" and c is not np.nan and '已退款' in e:
        return "人审拒绝"
    elif a is np.nan and b=="进件" and d is not np.nan:
        return "客户取消"
    #elif a is np.nan and b=="进件" and f is not np.nan:
        #return "无法联系"
    elif g is not np.nan and b=="进件" and "蚂蚁数控风险等级" in g:
        return "出库前风控强拒"
    elif g is not np.nan and b=="进件" and "命中出库前风控流强拒" in g:
        return "出库前风控强拒"
    elif a is np.nan and b=="进件" and "待审核" in e:
        return "待审核"
    elif a is np.nan and b=="进件" and "待发货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "待收货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "租赁中" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    else :
        return e
    
    

In [220]:
df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["订单状态"],x["订单状态"],x["备注信息"]),axis=1)

In [221]:

fw_path = 'F:/myfile/p站数据/服务订单'
fw_excel_names = []
fw_excel_date = []
for excel_name in os.listdir(fw_path):
    fw_excel_names.append(excel_name)
    fw_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(fw_excel_date)


['~$服务订单0611-0617', '服务订单1001-1015']


In [222]:
f_path_insure=f"F:/myfile/p站数据/服务订单/{fw_excel_date[-1]}.xlsx"
df_insure=pd.read_excel(f_path_insure,skiprows=0)
df_insure1=pd.merge(df,df_insure,left_on="单号", right_on="租赁订单号",how="left")
df=df_insure1

In [223]:
import datetime

Today=datetime.date.today()
str(Today)
Now=str(datetime.datetime.now() )
Now

'2025-10-17 15:22:17.781312'

# 删除商家数据

In [224]:
df_contain = df.copy()
# df_contain[df_contain['商家']=='星晟数码'].shape
# 选取商家为人人享租，喜卓灵新租机的订单数据，宏生需求，2025-09-11
# df_ = df[df['商家'].isin(['人人享租','喜卓灵新租机'])]
# df_.商家.unique()

## 获取出库类台账数据（宏生）

In [225]:
# import pymysql
# def query(sql,
#         host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
#         user="ylc",
#         password="1O8t5lcJ5aMhwwPEUUjS",
#         database = '' ,
#         port=3306
#         ):   
#     conn = pymysql.connect(
#         host=host,
#         user=user,
#         port=port,
#         password=password,
#         max_allowed_packet=1073741824,
#         charset="utf8")
#     try:
#         df = pd.read_sql(sql, con=conn)
#         conn.close()
#     except:
#         print('error')
#         conn.close()
#         raise
#     return df

In [226]:
# # 订单号  租金om.all_money	已付租金tos.real_pay_money(tos.status=3)	期数tos.sort  碎屏险（是否有碎屏险）buy_service_product，1否2是

# sql_ck = '''
# select om.id order_id,om.order_number 订单号, om.all_money, tos.real_pay_money, tos.sort, om.buy_service_product,om.status

# from db_digua_business.t_order om
# left join db_digua_business.t_order_stages tos on om.id=tos.order_id
# where om.user_mobile is not null 
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-08-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-10'
# and tos.status=3
# '''
# df_ck = query(sql_ck)

# df_ck.shape

## 去重

In [227]:
# # 按照order_id、sort去重
# duplicates = df_ck[df_ck.duplicated(subset=['order_id', 'sort'], keep=False)]
# if not duplicates.empty:
#     print("发现重复数据，数量：", len(duplicates))
#     # 可以选择保留最新的记录（根据实际业务逻辑调整）
#     # 这里假设reality_refund_date不为空的是更新的记录
#     df_ck = df_ck.sort_values(['order_id', 'sort', 'refund_date'], ascending=[True, True, False])
#     # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
#     df_ck = df_ck.drop_duplicates(subset=['order_id', 'sort'], keep='first')
    
# df_ck["是否出库"] = np.where(df_ck["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
# df_ck["是否有碎屏险"] = np.where(df_ck["buy_service_product"] == 2, "有碎屏险", "无碎屏险")
# df_ck.loc[:,"已付租金"]=df_ck["real_pay_money"].fillna(0)
# df_ck.loc[:,"期数"] = df_ck["sort"]
# df_ck.loc[:,"总租金"]=df_ck["all_money"]
# df_ck = df_ck[df_ck['是否出库'] == 1][['order_id', '订单号', '总租金', '已付租金', '期数', '是否有碎屏险']]
# df_ck.columns

In [228]:

# dfck=pd.merge(df_ck,df_,left_on="订单号", right_on="单号")
# dfck.shape


In [229]:

df.drop(df[df['商家']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="优优2店"].index,inplace=True)
df.drop(df[df['商家']=="探路者自营账户"].index,inplace=True)
df.drop(df[df['商家']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['商家']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['商家']=="北京海鸟窝科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="澄心优租"].index,inplace=True)
df.drop(df[df['商品型号'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
# 趣智数码  单
df.drop(df[df['商家']=="趣智数码"].index,inplace=True)
df.drop(df[df['商家']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['商家']=="广州康基贸易有限公司"].index,inplace=True)
df.drop(df[df['商家']=="汇客好租"].index,inplace=True)
df.drop(df[df['商家']=="线下小店"].index,inplace=True)
df.drop(df[df['商家']=="乙辉数码"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="南京聚格网络科技"].index,inplace=True)
df.drop(df[df['商家']=="呱子笔记本电脑"].index,inplace=True)

df.drop(df[df['商家'] == "星晟数码"].index, inplace=True)
df.drop(df[df['商家'] == "蘑菇时间"].index, inplace=True)
# df = clean.drop_merchant(df)

In [230]:
df_j=df[df["进件"]=="进件"]

In [231]:
# 商家拒量
df.drop(df[df['商家']=="小蚂蚁租机"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="人人享租"].index,inplace=True)
df.drop(df[df['商家']=="崇胜数码"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵租机"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵新租机"].index,inplace=True)

In [232]:
df_j[df_j.归属渠道=='搜索渠道'].shape

(6489, 88)

# 以下为出库相关数据，要统计到出库相关的数据，必须运行到以下读取出库台账的代码

In [233]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023",skiprows=0) 
f_path_ck="F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck=pd.read_excel(f_path_ck,sheet_name="2025",skiprows=0)
dfck=pd.merge(df_ck,df,left_on="订单号", right_on="单号")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape


(2021, 106)

In [234]:
dfck.订单状态.value_counts()

订单状态
租赁中    1691
待收货     251
已退款      39
已完成      27
待发货      13
Name: count, dtype: int64

# 检查出库订单中有没有客户取消的单，即订单状态为”已退款"的单，找出来删除后重新运行上一块导入出库台账的代码

In [235]:
dfck.drop(dfck[dfck["订单状态"]=="已退款"].index,inplace=True)

# 剔除拒收!!!!
dfck.drop(dfck[dfck["订单状态"]=="已完成"].index,inplace=True)

In [236]:
dfck.订单状态.value_counts()

订单状态
租赁中    1691
待收货     251
待发货      13
Name: count, dtype: int64

In [237]:
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)

# 苹果15系列商品型号-进件

In [238]:
# #苹果15系列机型进件
# df_j["型号内存"]=df_j["商品型号"]+"_"+df_j["内存"]
# df_jap15=df_j[df_j["型号内存"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["型号内存"],margins=True).to_excel(f"F:苹果15进件.xlsx")

In [239]:
# #华为进件
# df_jhw=df_j[df_j["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# pd.crosstab(df_jhw["商品型号"],df_jhw["内存"],margins=True)


In [240]:
# df_jhw.groupby(["商品型号","内存","颜色"]).agg({"单号":np.size}).unstack(-1)

# 苹果15系列商品型号-出库

In [241]:
# # 苹果15系列商品型号-出库
# df_ap15=dfck[dfck["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True)

In [242]:
# #华为出库
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]

# pd.crosstab(dfckhw["商品型号"],dfckhw["内存_x"],margins=True)

In [243]:
# dfckhw.groupby(["商品型号","内存_y","颜色_x"]).agg({"订单号":np.size}).unstack(-1)

# 某渠道苹果15系列商品型号-进件

In [244]:
#苹果15系列商品型号-去重订单
# df_qd=df_j[df_j["来源渠道"]=="芝麻租物"]
# df_jap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["商品型号"],margins=True).to_excel("F:/芝麻租物/2024.3芝麻渠道15进件.xlsx")

# 某渠道苹果15系列商品型号-去重订单


In [245]:
# # 某渠道苹果15系列商品型号-进件
# df_qd=df[df["来源渠道"]=="芝麻租物"]
# df_ap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15去重订单.xlsx")

# 某渠道苹果15系列商品型号-出库

In [246]:
# # 某渠道苹果15系列商品型号-出库
# dfck_qd=dfck[dfck["来源渠道"]=="芝麻租物"]
# dfck_jap15=dfck_qd[dfck_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(dfck_jap15["下单日期"],dfck_jap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15出库.xlsx")

# 宏生碎屏险数据，导出来直接甩给他就可以了

In [247]:
# #总体出库订单碎屏险购买数据
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
# df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"] #+df_s2merge["待支付"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库"]/df_s2merge["合计出库"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
# df_s2merge

In [248]:
# #芝麻租物出库订单碎屏险购买数据
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]+df_zws2merge["待确认"]#+df_s2merge["待支付"]
# df_zws2merge

In [249]:
# #Mate60Pro 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60Pro",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]#+df_zws2merge["待确认"]
# df_zws2merge

In [250]:
# #Mate60 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# # df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]   # df_zws2merge["待确认"]+
# df_zws2merge

# 总体碎屏险数据

In [251]:
# #总体碎屏险数据,宏生需求
# df_bx=pd.crosstab(df["下单日期"],df["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx=df_bx.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单",})
# df_bx_j=pd.crosstab(df_j["下单日期"],df_j["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_j=df_bx_j.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件",})
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_spx1=pd.merge(df_bx,df_bx_j,left_index=True, right_index=True)
# df_spx2=pd.merge(df_spx1,df_bx_ck,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/总体碎屏险数据{excel_date[-1]}.xlsx")

In [252]:
# df_s1=pd.crosstab(df["下单日期"],df["服务订单状态"],margins=True)
# df_s1=df_s1.rename(columns={"All":"合计购买碎屏险订单"})
# df_s2=pd.crosstab(df_j["下单日期"],df_j["服务订单状态"],margins=True)
# df_s2=df_s2.rename(columns={"All":"合计购买碎屏险进件"})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.rename(columns={"All":"合计购买碎屏险出库"})
# df_s1merge=pd.merge(df_s1,df_s2,left_index=True, right_index=True)
# df_s2merge=pd.merge(df_s1merge,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"]
# df_s2merge

# 芝麻租物碎屏险数据

In [253]:
# #芝麻租物碎屏险数据,宏生需求
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zw1=pd.crosstab(dfzw["下单日期"],dfzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw1=df_zw1.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单"})

# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zw2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw2=df_zw2.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件"})
# df2=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(df2["下单日期"],df2["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zwspx1=pd.merge(df_zw1,df_zw2,left_index=True, right_index=True)
# df_zwspx2=pd.merge(df_zwspx1,df_zw3,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/芝麻租物碎屏险数据{excel_date[-1]}.xlsx")

In [254]:
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zws1=pd.crosstab(dfzw["下单日期"],dfzw["服务订单状态"],margins=True)
# df_zws1=df_zws1.rename(columns={"All":"合计购买碎屏险订单"})
# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zws2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["服务订单状态"],margins=True)
# df_zws2=df_zws2.rename(columns={"All":"合计购买碎屏险进件"})
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.rename(columns={"All":"合计购买碎屏险出库"})
# df_zws1merge=pd.merge(df_zws1,df_zws2,left_index=True, right_index=True)
# df_zws2merge=pd.merge(df_zws1merge,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=+df_zws2merge["服务中"]+df_zws2merge["待确认"]
# df_zws2merge

In [255]:
#dfck[dfck["是否购买服务商品：1、否；2、是"]==2].to_excel(f"C:/Users/Administrator/Desktop/总体碎屏险数据出库_1119.xlsx")

# 月中\月底运营数据复盘 

# 月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比

In [256]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,来源渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,10,京东出游季,30,7134.087333,13163.300000,10199.000000,69.95%
1,10,八派信息,3,10996.896667,12996.700000,9999.000000,109.98%
2,10,其他渠道场景渠道。,3,6620.386667,12996.700000,9999.000000,66.21%
3,10,单人聊天会话中的小程序消息卡片（分享）,2,6997.290000,12996.700000,9999.000000,69.98%
4,10,我的小程序入口,7,8634.035714,12484.134286,9480.000000,91.08%
5,10,搜索-租手机,24,7397.342917,13213.333333,10165.666667,72.77%
6,10,搜索-租苹果手机,20,7317.704000,12914.716000,9899.000000,73.92%
7,10,搜索下挂-商品,146,6917.215616,12859.187123,9889.410959,69.95%
8,10,搜索侠客行-核心词,4,9422.207500,11621.935000,8649.000000,108.94%
9,10,搜索侠客行-苹果17,4,10913.812500,14211.530000,10749.000000,101.53%


In [257]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,归属渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,10,京东出游季,30,7134.087333,13163.300000,10199.000000,69.95%
1,10,京享租,446,8568.760493,12914.047309,9990.479821,85.77%
2,10,八派信息,3,10996.896667,12996.700000,9999.000000,109.98%
3,10,其他渠道场景渠道。,2,6997.290000,12996.700000,9999.000000,69.98%
4,10,单人聊天会话中的小程序消息卡片（分享）,2,6997.290000,12996.700000,9999.000000,69.98%
5,10,我的小程序入口,6,8767.041667,12280.156667,9232.333333,94.96%
6,10,抖音渠道,1,7836.000000,13708.000000,10966.000000,71.46%
7,10,搜索渠道,583,7246.974443,12998.947684,9986.478559,72.57%
8,10,支付宝客户端首页,15,7341.583333,11770.900000,8859.000000,82.87%
9,10,支付宝直播,2,7731.690000,14776.460000,10999.000000,70.29%


# 月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比

In [258]:
dfck_all=dfck.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,10,1955,6998.261442,12885.468726,9954.123785,9954.123785,70.31%


In [259]:
#月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比
dfck_fjd = dfck[dfck.归属渠道!='京享租']
dfck_all=dfck_fjd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,10,1509,6534.084785,12877.022041,9943.378396,9943.378396,65.71%


# 京享租

In [260]:
dfck_jd = dfck[dfck.归属渠道=='京享租']
dfck_all=dfck_jd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,10,446,8568.760493,12914.047309,9990.479821,9990.479821,85.77%


# 月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比

In [261]:
#月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,10,租完即送,31,13062.891613,13062.891613,9950.612903,9950.612903,131.28%
1,10,租完归还,1478,6397.147700,12873.123559,9943.226658,9943.226658,64.34%


# 京享租

In [262]:
dfck_all=dfck_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,10,租完即送,53,12948.647170,12948.64717,9999.000000,9999.000000,129.50%
1,10,租完归还,393,7978.088753,12909.38117,9989.330789,9989.330789,79.87%


# 月中月底各渠道各租赁方案出库数

In [263]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["来源渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
来源渠道,,,
京东出游季,2,28,30
八派信息,2,1,3
其他渠道场景渠道。,0,3,3
单人聊天会话中的小程序消息卡片（分享）,0,2,2
我的小程序入口,2,5,7
搜索-租手机,1,23,24
搜索-租苹果手机,1,19,20
搜索下挂-商品,0,146,146
搜索侠客行-核心词,2,2,4


In [264]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["归属渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
归属渠道,,,
京东出游季,2,28,30
京享租,53,393,446
八派信息,2,1,3
其他渠道场景渠道。,0,2,2
单人聊天会话中的小程序消息卡片（分享）,0,2,2
我的小程序入口,2,4,6
抖音渠道,0,1,1
搜索渠道,21,562,583
支付宝客户端首页,2,13,15


# 月中月底各渠道租赁方案进件数

In [265]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
来源渠道,,,
九州信息,5,0,5
京东出游季,10,387,397
付费流量(通过商家数字推广平台，灯火等投放的广告),1,3,4
八派信息,469,38,507
公域搜索带券,2,0,2
其他渠道场景渠道。,11,111,122
单人聊天会话中的小程序消息卡片（分享）,11,848,859
卡包\r\n,0,2,2
宜品花,24,0,24


In [266]:
df_zulin = pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)
# df_zulin.to_excel(f'F:/各渠道租赁进件数/{excel_date[-1]}进件数.xlsx')

In [267]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["归属渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
归属渠道,,,
九州信息,5,0,5
京东出游季,10,368,378
京享租,109,9954,10063
付费流量(通过商家数字推广平台，灯火等投放的广告),1,2,3
八派信息,469,35,504
其他渠道场景渠道。,11,76,87
单人聊天会话中的小程序消息卡片（分享）,11,840,851
卡包\r\n,0,2,2
宜品花,24,0,24


# 以下代码为测算毛利时候使用，使用前先从M站系统导出出库单获取成本数据，然后按顺序运行以下代码

In [268]:

ck_path = 'F:/myfile/p站数据/出库单'
ck_excel_names = []
ck_excel_date = []
for excel_name in os.listdir(ck_path):
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        ck_excel_names.append(excel_name)
        ck_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(ck_excel_date)
os.listdir(ck_path)

['10月出库']


['10月出库.xlsx', '~$7月出库.xlsx']

In [295]:
f_path_cost=f"F:/myfile/p站数据/出库单/{ck_excel_date[-1]}.xlsx"
df_cost=pd.read_excel(f_path_cost,sheet_name='出库表',skiprows=0)
#df_cost.loc[:,"成本"]=np.where(df_cost["供应商"].str.contains('侯立')|df_cost["供应商"].str.contains('张陵')|df_cost["供应商"].str.contains('拾贝壳')|df_cost["供应商"].str.contains('大龙发'),(df_cost["设备成本"]-10)/1.025,df_cost["设备成本"])
# df_cost["成本"]=df_cost["设备成本"]
# df_cost["成本"]=df_cost["成本"].round()
dfck_cost=pd.merge(dfck,df_cost,left_on="订单号", right_on="订单号")
# 更改原因：由于台账存在成本字段与出库单表的成本字段发生冲突
dfck_cost.loc[:, "成本"]=np.where(dfck_cost["设备成本"] is np.nan, dfck_cost["成本"],dfck_cost["设备成本"])
dfck_cost["成本"]=dfck_cost["成本"].round()
dfck_cost.drop_duplicates(subset=["订单号"],inplace=True)
# 宏生需求
# dfck_cost.to_excel(f"F:/毛利/明细_{Today}.xlsx")

In [270]:
# 毛利计算过程，这块代码不能忽略
dfck_cost.loc[:,"原始毛利"]=dfck_cost["买断金额"]-dfck_cost["成本"]+dfck_cost["商品溢价费"]
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),"苹果","非苹果")
dfck_cost["商品型号_all"] = dfck_cost["商品型号"][dfck_cost["商品型号"].str.contains('iP')==True] 
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),  
#                                 np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16", "苹果15"),"非苹果")
dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),
                                np.where(dfck_cost["商品型号_all"].str.contains(r'iPhone 17|iPhone Air'), "苹果17",
                                    np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16",
                                            np.where(dfck_cost['商品型号_all'].str.contains('15'), "苹果15",
                                                    np.where(dfck_cost['商品型号_all'].str.contains('14'), "苹果14", '苹果14以下')))),"非苹果")
'''
2025-8及以前的残值
def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果16":
        return s["成本"]*0.6
    elif s["残值机型"]=="苹果15":
        return s["成本"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["成本"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["成本"]*0.375
    else:
        return s["成本"]*0.425
'''


def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果17":
        return s["总押金"]*0.60
    elif s["残值机型"]=="苹果16":
        return s["成本"]*0.45
    elif s["残值机型"]=="苹果15":
        return s["成本"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["成本"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["成本"]*0.375
    else:
        return s["成本"]*0.425
dfck_cost.loc[:,"残值"]=dfck_cost.apply(getrecanzhi,axis=1)
dfck_cost.loc[:,"总应收"]=dfck_cost["总租金"]+dfck_cost["残值"]+dfck_cost["商品溢价费"]
dfck_cost.loc[:,"残值估算毛利"]=dfck_cost["总应收"]-dfck_cost["成本"]
def getremark(s):
    if s["成本"]==0:
        return s["已付金额"]
    else:
        return s["残值估算毛利"]
dfck_cost["残值估算毛利"]=dfck_cost.apply(getremark,axis=1)
dfck2 = dfck_cost[dfck_cost["成本"].notnull()]

In [271]:
# dfck_cost.groupby('月份').agg({'订单号': 'count', '总租金': 'sum', '残值': 'sum', '商品溢价费': 'sum', '总应收': 'sum', '成本': 'sum', '残值估算毛利': 'mean', '买断金额': 'sum', '原始毛利': 'mean'})

# 月中、月底总体数据测算毛利

In [272]:
# 月中、月底总体数据测算毛利
# 总应收=总租金+残值+溢价费
# 残值估算毛利=总应收-成本
# 买断估算毛利(原始毛利)=买断金额-成本+溢价费
# 租售比=总租金均值/商品押金均值
dfck2_fjd = dfck2[dfck2.归属渠道!='京享租']
dfck_zsb=dfck2_fjd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,1487,9205.573255,12878.706348,3690.91924,1428.67965,15.52%,40.09%


In [273]:
dfck2_jd = dfck2[dfck2.归属渠道=='京享租']
dfck_zsb=dfck2_jd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,443,9237.677201,12914.151242,3700.011287,3041.686005,32.93%,40.05%


# 租物和非租物

In [274]:
# # 月中、月底总体数据测算毛利
# dfck2_zw = dfck2[dfck2.归属渠道=='芝麻租物']
# dfck_zsb=dfck2_zw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb


In [275]:
# dfck2_fzw = dfck2[dfck2.归属渠道!='芝麻租物']
# dfck_zsb=dfck2_fzw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb

# 月中、月底分租赁方案测算毛利

In [276]:
dfck_zsb=dfck2.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,租完即送,84,9354.666667,12990.808810,3701.999286,3701.999286,39.57%,39.57%
1,10,租完归还,1846,9206.493191,12882.111268,3692.59695,1712.321560,18.60%,40.11%


In [277]:
#月中、月底分租赁方案测算毛利
dfck_zsb=dfck2_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,租完即送,31,9459.16129,13062.891613,3771.246452,3771.246452,39.87%,39.87%
1,10,租完归还,1456,9200.174059,12874.784821,3689.208977,1378.803571,14.99%,40.10%


# 京享租

In [278]:
dfck_zsb=dfck2_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,租完即送,53,9293.54717,12948.647170,3661.496226,3661.496226,39.40%,39.40%
1,10,租完归还,390,9230.084615,12909.463333,3705.245385,2957.455385,32.04%,40.14%


# 月中、月底分渠道测算毛利

In [279]:
# 月中、月底分渠道测算毛利
dfck_zsb=dfck2.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,来源渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,京东出游季,29,9349.034483,13172.424138,3823.389655,1720.067931,18.40%,40.90%
1,10,八派信息,3,9190.0,12996.700000,3806.7,3195.896667,34.78%,41.42%
2,10,其他渠道场景渠道。,3,9351.666667,12996.700000,3645.033333,1476.970000,15.79%,38.98%
3,10,单人聊天会话中的小程序消息卡片（分享）,2,9175.0,12996.700000,3821.7,1951.040000,21.26%,41.65%
4,10,我的小程序入口,7,8947.061429,12484.134286,3685.358571,3068.188571,34.29%,41.19%
5,10,搜索-租手机,24,9346.125,13213.333333,3867.208333,2083.724167,22.30%,41.38%
6,10,搜索-租苹果手机,20,9192.5,12914.716000,3752.166,2165.299000,23.56%,40.82%
7,10,搜索下挂-商品,145,9133.965517,12858.238759,3729.093931,1906.634966,20.87%,40.83%
8,10,搜索侠客行-核心词,4,8731.0,11621.935000,3315.435,2767.007500,31.69%,37.97%
9,10,搜索侠客行-苹果17,4,10168.75,14211.530000,4392.53,3978.600000,39.13%,43.20%


In [280]:
dfck_zsb=dfck2.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,归属渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,京东出游季,29,9349.034483,13172.424138,3823.389655,1720.067931,18.40%,40.90%
1,10,京享租,443,9237.677201,12914.151242,3700.011287,3041.686005,32.93%,40.05%
2,10,八派信息,3,9190.0,12996.700000,3806.7,3195.896667,34.78%,41.42%
3,10,其他渠道场景渠道。,2,9282.5,12996.700000,3714.2,1891.915000,20.38%,40.01%
4,10,单人聊天会话中的小程序消息卡片（分享）,2,9175.0,12996.700000,3821.7,1951.040000,21.26%,41.65%
5,10,我的小程序入口,6,9016.666667,12280.156667,3436.49,2598.525000,28.82%,38.11%
6,10,抖音渠道,1,8529.43,13708.000000,5178.57,5886.170000,69.01%,60.71%
7,10,搜索渠道,578,9232.16609,12998.967128,3787.882353,2064.406107,22.36%,41.03%
8,10,支付宝客户端首页,15,8587.8,11770.900000,3472.1,2649.055000,30.85%,40.43%
9,10,支付宝直播,2,10955.0,14776.460000,4520.96,3211.140000,29.31%,41.27%


In [281]:
dfck_zsb =dfck2[dfck2["归属渠道"]=="搜索渠道"].groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,10,578,9232.16609,12998.967128,3787.882353,2064.406107,22.36%,41.03%


# 商家数据

In [302]:

sql_merchant = """
select om.id as order_id, om.order_type
from db_digua_business.t_order om
where date_format(om.create_time,'%Y-%m-%d') >= '2025-10-01'
and date_format(om.create_time,'%Y-%m-%d') <= '2025-10-15'
and om.merchant_name in ('星晟数码', '汇客好租')
"""
df_merchant = clean.query(sql_merchant)
df_merchant.shape

(3723, 2)

In [297]:
df_contain.columns

Index(['订单id', '单号', '外部订单号', '商家', '下单用户', '下单手机', '下单用户身份证号', '用户认证情况',
       '开始时间', '结束时间', '订单类型', '租期', '订单逾期天数', '总租金', '总押金', '商品押金', '总保险',
       '总运费', '总优惠', '订单状态', '是否买断', '下单时间', '收货人', '收货电话', '预授权支付单号', '活动名称',
       '收货省名称_x', '收货市名称_x', '收货区县名称', '收货人详细地址_x', '收货地址', '发货快递', '快递单号',
       '发货时间', '下单备注', '商品型号', '商品id', '下单商品', '商品属性', '商品数量', '租金单价', '买断金额',
       '预授权占用自有资金', '来源渠道', '备注信息', '用户实际支付金额', '联系人', '是否购买服务商品：1、否；2、是',
       '租物等级', '决策结果', '订单完成时间', '商品溢价费', '设置买断金额', '审核状态', '是否为资方订单',
       '第一期实付金额', '第二期应付金额', '第三期应付金额', '下单日期', '月份', '拒绝理由', '取消原因', '电审拒绝原因',
       '商品类型', '租赁方案', '押金类型', '是否使用优惠券', '是否进行预授权', '内存', '颜色', '进件', '归属渠道',
       '状态编码', '服务订单号', '租赁订单号', '服务商品名称', '服务商品价格', '服务商品改后价格', '服务订单状态',
       '服务时长（0表示长期）', '服务开始日期', '服务结束日期', '收货人姓名', '收货人电话', '收货省名称_y',
       '收货市名称_y', '区县名称', '收货人详细地址_y'],
      dtype='object')

In [303]:
# 去重订单可能会与日报数据存在些许出路，这是因为该用户在多个商家或渠道下过订单，去重的时候仅会保留一个订单
merchant_list = ['澄心优租', '优优2店', '北京海鸟窝科技有限公司', '租着用电脑数码', '趣智数码', '汇客好租(支付宝)', '汇客好租(抖音)', '乙辉数码','小蚂蚁租机', '兴鑫兴通讯', '呱子笔记本电脑', '南京聚格网络科技', '人人享租', '喜卓灵租机', '喜卓灵新租机', '星晟数码(支付宝)', '星晟数码(京东)', '蘑菇时间','云启德曜']
df_merge = df_contain.merge(df_merchant, left_on='订单id', right_on='order_id', how='left')
df_merge['商家'] = np.where((df_merge.商家=='星晟数码')&(df_merge.order_type=='ZFB_ORDER'), '星晟数码(支付宝)',
                        np.where((df_merge.商家=='星晟数码')&(df_merge.order_type=='JD_ORDER'), '星晟数码(京东)',
                            np.where((df_merge.商家=='汇客好租')&(df_merge.order_type=='ZFB_ORDER'), '汇客好租(支付宝)',
                                np.where((df_merge.商家=='汇客好租')&(df_merge.order_type=='DY_ORDER'), '汇客好租(抖音)', df_merge.商家))))               
df_merchant = df_merge[df_merge.商家.isin(merchant_list)]
# df_merchant = df_contain[df_contain.商家.isin(merchant_list)]
df_merchant.loc[:, '是否进件'] = np.where(df_merchant.进件=='进件', 1, 0)
df_merchant.loc[:, '出库'] = np.where(df_merchant.审核状态=='出库', 1, 0)
df_merchant_group = df_merchant.groupby(['月份', '商家']).agg({'单号': 'count', '是否进件': 'sum', '出库': 'sum', '总租金': 'mean', '买断金额': 'mean'}).rename(columns={'单号': '去重订单数', '是否进件': '进件数', '总租金': '租金平均价', '买断金额': '买断平均价'})
df_merchant_group.loc[:, '进件出库率'] = (df_merchant_group.出库/df_merchant_group.进件数).map(lambda x: format(x,".2%"))
df_merchant_group = df_merchant_group.reset_index()
df_merchant_group[['月份', '商家', '去重订单数', '进件数', '出库', '进件出库率', '租金平均价', '买断平均价']]

,月份,商家,去重订单数,进件数,出库,进件出库率,租金平均价,买断平均价
0,10,云启德曜,371,371,16,4.31%,6420.696685,12990.501456
1,10,人人享租,653,641,16,2.50%,7182.854809,12848.008729
2,10,北京海鸟窝科技有限公司,5,1,0,0.00%,326.800000,2400.000000
3,10,喜卓灵新租机,933,932,43,4.61%,7342.251779,12986.947588
4,10,星晟数码(京东),961,842,29,3.44%,3932.586420,8055.020083
5,10,星晟数码(支付宝),1522,183,9,4.92%,4495.046919,8600.135821
6,10,汇客好租(支付宝),46,8,0,0.00%,4818.614130,9584.521739
7,10,澄心优租,15,2,0,0.00%,3174.333333,5378.133333
8,10,租着用电脑数码,120,41,2,4.88%,2092.600250,4793.533333
9,10,蘑菇时间,38,17,1,5.88%,995.239211,4445.215789


In [301]:
# 方案一：设置na参数，将NaN值视为False
df_merge[df_merge.商家.str.contains('星晟数码', na=False)]

,订单id,单号,外部订单号,商家,下单用户,下单手机,下单用户身份证号,用户认证情况,开始时间,结束时间,...,服务开始日期,服务结束日期,收货人姓名,收货人电话,收货省名称_y,收货市名称_y,区县名称,收货人详细地址_y,order_id,order_type
19,4094272,A20251001234349160,NaN,星晟数码,李志博,15176235304,130623200701315818,未认证,2025-10-04,2026-10-04,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,4094238,A20251001233337126,NaN,星晟数码,赵紫龙,16606141736,511381199808208573,未认证,2025-10-04,2026-10-04,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,4094209,A2025100123231498,NaN,星晟数码,汪明锋,19944034776,360122199006090315,未认证,2025-10-04,2026-10-04,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,4094030,A20251001223537122,NaN,星晟数码,宋汶聪,13283373513,130828200708282716,未认证,2025-10-04,2026-10-04,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,4094023,A20251001223318116,NaN,星晟数码,李少聪,15288596053,532324200709222313,未认证,2025-10-04,2026-10-04,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83745,4219459,A202510151550241019,NaN,星晟数码,杨平,15280953180,510722199606164200,未认证,2025-10-18,2026-10-18,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83835,4218575,A20251015150539107,NaN,星晟数码,余浩,19936385441,500230200210146838,已认证,2025-10-18,2026-10-18,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83841,4218235,A20251015144805866,NaN,星晟数码,张新坤,13722530567,130283199405075679,已认证,2025-10-18,2026-10-18,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83844,4217981,A20251015143357603,NaN,星晟数码,李建辉,15998081438,211122200110180517,已认证,2025-10-18,2026-10-18,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
# df_merchant_xcsm = df_contain[df_contain['商家']=='星晟数码']
# df_merchant_xcsm.loc[:, '是否进件'] = np.where(df_merchant_xcsm.进件=='进件', 1, 0)
# df_merchant_xcsm.loc[:, '出库'] = np.where(df_merchant_xcsm.审核状态=='出库', 1, 0)
# df_group = df_merchant_xcsm.groupby('下单日期').agg({'单号': 'count', '是否进件': 'sum', '出库': 'sum', '总租金': 'mean', '买断金额': 'mean'}).rename(columns={'单号': '去重订单数', '是否进件': '进件数', '总租金': '租金平均价', '买断金额': '买断平均价'})
# df_group

In [286]:
# df1 = pd.read_excel('F:/myfile/p站数据/当月数据/0801-0815.xlsx')
# # df0 =df1.copy()
# df1.shape

In [287]:
# df0_xcsm = df1[df1['商家']=='星晟数码']
# df0_xcsm["下单日期"]=df0_xcsm["下单时间"].dt.date
# df0_xcsm["下单日期"]=pd.to_datetime(df0_xcsm["下单日期"],errors="coerce")
# df0_xcsm["下单日期"]
# df0_xcsm["月份"]=df0_xcsm["下单日期"].dt.month
# #删除重复单号
# df0_xcsm.drop_duplicates(subset=["单号"],inplace=True)
# #删除身份证空值行
# df0_xcsm.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
# #去刷单订单
# df0_xcsm.drop(df0_xcsm[df0_xcsm['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单秘密计划"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单曙光计划"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="线下门店3个月试行"].index,inplace=True)
# #删除重复单号
# df0.drop_duplicates(subset=["单号"],inplace=True)
# #删除订单状态空值行
# df0_xcsm.dropna(subset=["订单状态"],axis=0,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)
# df0_xcsm.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)
# df0_xcsm.单号.count()

In [288]:
# df_xcsm_list = df0_xcsm1.单号.to_list()
# len(df_xcsm_list)

In [289]:
# df0 =df1.copy()
# df0["下单日期"]=df0["下单时间"].dt.date
# df0["下单日期"]=pd.to_datetime(df0["下单日期"],errors="coerce")
# df0["下单日期"]
# df0["月份"]=df0["下单日期"].dt.month


In [290]:
# #删除重复单号
# df0.drop_duplicates(subset=["单号"],inplace=True)
# #删除身份证空值行
# df0.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
# #去刷单订单
# df0.drop(df0[df0['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单秘密计划"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单曙光计划"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="线下门店3个月试行"].index,inplace=True)
# #删除订单状态空值行
# df0.dropna(subset=["订单状态"],axis=0,inplace=True)

In [291]:
# df0.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)


In [292]:
# df0.drop(df0[df0['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)



# 机型均值

In [293]:
# # 商品押金  签约价   采购价   
dfck2["型号内存"]=dfck2["商品型号"]+"_"+dfck2["内存_x"]
dfck2.groupby(['型号内存']).agg({'商品押金':'mean','买断金额':'mean','成本':'mean'})

,商品押金,买断金额,成本
型号内存,,,
Apple Watch Ultra 3_GPS+蜂窝网络49mm,6499.000000,8447.400000,6355.0
REDMI K80_16GB+256GB,2499.000000,3248.200000,2687.0
iPad Pro 2024款_256GBWiFi版,8999.000000,11696.900000,7554.0
iPhone 17 Pro Max_256GB,9999.000000,13796.620000,10675.25
iPhone 17 Pro Max_512GB,11999.000000,16556.220000,12518.571429
iPhone 17 Pro_256GB,8999.000000,12296.983333,9181.666667
iPhone 17 Pro_512GB,10999.000000,15176.420000,11280.0
iPhone 17_256GB,5999.000000,8277.420000,6345.0
iPhone16ProMax_256GB,10000.148106,12923.455396,9255.903559


In [294]:
# df_zulin = dfck2.groupby(['型号内存']).agg({'订单号':'count', '商品押金':'mean','买断金额':'mean','成本':'mean'})
# df_zulin.to_excel(f'F:/宏生/{excel_date[-1]}宏生转化金额.xlsx')